In [1]:
import pandas as pd      
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
import warnings
warnings.filterwarnings('ignore')
import xgboost

In [2]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    mse = mean_squared_error(actual, pred)
    score = r2_score(actual, pred)
    return print("r2_score:", score, "\n","mae:", mae, "\n","mse:",mse, "\n","rmse:",rmse)

In [3]:
pwd

'C:\\Users\\ozlem\\Cway\\projects\\autscout_price_prediction'

In [4]:
df=pd.read_pickle("golden_data_not_dummy.pkl")
pd.set_option('display.max_columns', None)

In [5]:
new_list=["age", "hp", "km", "model"]

In [6]:
X=df[new_list]
y=df['price']
X=pd.get_dummies(X)

In [7]:
X.head()

,age,hp,km,model_A1,model_A2,model_A3,model_Astra,model_Clio,model_Corsa,model_Espace,model_Insignia
0,3,66.0,56013,1,0,0,0,0,0,0,0
1,2,141.0,80000,1,0,0,0,0,0,0,0
2,3,85.0,83450,1,0,0,0,0,0,0,0
3,3,66.0,73000,1,0,0,0,0,0,0,0
4,3,66.0,16200,1,0,0,0,0,0,0,0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [9]:
model = xgboost.XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [10]:
y_pred = model.predict(X_test)
eval_metrics(y_test, y_pred)

r2_score: 0.9113619721434223 
 mae: 1411.5892848071753 
 mse: 4628539.5378820915 
 rmse: 2151.4040852155345


In [11]:
from sklearn.ensemble import RandomForestRegressor
rf_reg=RandomForestRegressor()
rf_reg.fit(X_train, y_train)

RandomForestRegressor()

In [12]:
y_pred2 = rf_reg.predict(X_test)

In [13]:
eval_metrics(y_test,y_pred2)

r2_score: 0.8985323107600275 
 mae: 1445.2657639429945 
 mse: 5298484.440839169 
 rmse: 2301.84370469395


In [14]:
import pickle
import pandas as pd
pickle.dump(rf_reg, open('rf_model.pkl', 'wb'))
pickle.dump(model, open('xgb_model.pkl', 'wb'))

**deployment**

In [17]:
richard_model = pickle.load(open('xgb_model.pkl', 'rb'))

In [18]:
my_dict = {
    "age": 2,
    "hp": 105,
    "km": 100000,
    "model": 'Astra'
}

df = pd.DataFrame.from_dict([my_dict])

In [19]:
df

,age,hp,km,model
0,2,105,100000,Astra


In [20]:
columns=list(X.columns)
columns

['age',
 'hp',
 'km',
 'model_A1',
 'model_A2',
 'model_A3',
 'model_Astra',
 'model_Clio',
 'model_Corsa',
 'model_Espace',
 'model_Insignia']

In [21]:
df = pd.get_dummies(df).reindex(columns=columns, fill_value=0)

In [22]:
df.head()

,age,hp,km,model_A1,model_A2,model_A3,model_Astra,model_Clio,model_Corsa,model_Espace,model_Insignia
0,2,105,100000,0,0,0,1,0,0,0,0


In [23]:
prediction = richard_model.predict(df)
print(prediction)

[13935.282]


In [24]:
print("The estimated price of your car is €{}. ".format(int(prediction[0])))

The estimated price of your car is €13935. 
